In [ ]:
import gymnasium as gym
import os
import source.config as config
from source.algorithms import *
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import source.farm_env # Import necessary to register the gym environment

In [ ]:
env = gym.make(id='FarmEnv-v0',
                initial_budget = config.INITIAL_BUDGET,
                sheep_cost = config.SHEEP_COST,
                wheat_cost = config.WHEAT_COST,
                wool_price = config.WOOL_PRICE,
                wheat_price = config.WHEAT_PRICE,
                max_years = config.MAX_YEARS,
                wool_fixed_cost = config.WOOL_FIXED_COST,
                storm_probability = config.STORM_PROBABILITY,
                incest_penalty = config.INCEST_PENALTY,
                reward_std = config.SIGMA
                )

In [ ]:
def conv(a, win=100):
    return np.convolve(a, np.ones(win), mode='same') / win

In [ ]:
learning_rate = 0.001
policy_learning_rate = 1e-4
value_learning_rate = 1e-4
n_episodes = 500_000
start_epsilon = 1.0
epsilon_decay = start_epsilon / (n_episodes)  # reduce the exploration over time
final_epsilon = 0.1

REINFORCE_agent = FarmAgentNeuralREINFORCEAdvantage(
    environment=env, policy_learning_rate=policy_learning_rate, value_learning_rate=value_learning_rate, epsilon=start_epsilon, epsilon_decay=epsilon_decay, final_epsilon=final_epsilon, gamma=.999,
    # policy_net_weights_path='agent_models/policy_net_weights.pth', value_net_weights_path='agent_models/value_net_weights.pth'
)

MC_vfa_agent = FarmAgentMCVFA(
    environment=env, learning_rate=learning_rate, epsilon=start_epsilon, epsilon_decay=epsilon_decay, final_epsilon=final_epsilon, gamma=.999
)

SARSA_vfa_agent = FarmAgentSarsaVFA(
    environment=env, learning_rate=learning_rate, epsilon=start_epsilon, epsilon_decay=epsilon_decay, final_epsilon=final_epsilon, gamma=.95
)

# REINFORCE

In [ ]:
nenv = gym.wrappers.RecordEpisodeStatistics(env, deque_size=n_episodes)
final_budget_queue = []

for episode in tqdm(range(n_episodes)):
    REINFORCE_agent.update(episode_number=episode)
    REINFORCE_agent.decay_epsilon()
    final_budget_queue.append(nenv.unwrapped.budget)

    # Optionally, you can print some statistics every N episodes
    if episode % 10000 == 0:
        avg_budget = sum(final_budget_queue[-10000:]) / min(10000, len(final_budget_queue))
        print(f"Episode {episode}, Average Budget: {avg_budget:.2f}") #, Weights: {list(REINFORCE_agent.policy_net.parameters())}")

Statistics evaluation

In [ ]:
w = 1000
# rewards = conv(np.array(nenv.return_queue).flatten(), win=w)
# lengths = conv(np.array(nenv.length_queue).flatten(), win=w)
# error = conv(np.array(REINFORCE_agent.training_error).flatten(), win=w)
final_budgets = conv(np.array(final_budget_queue), win=w)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4), ncols=2)
sns.lineplot(x=range(len(np.array(0).flatten())), y=np.array(0).flatten(), ax=ax[0], c='#acc700', linewidth=.2)
ax[0].set_title('Error')
sns.lineplot(x=range(len(final_budget_queue)), y=final_budget_queue, ax=ax[1], c='#acc700', linewidth=.2)
ax[1].set_title('Final budget')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
sns.lineplot(x=range(len(final_budgets)), y=final_budgets, ax=ax, c='#acc700', linewidth=.2)
ax.set_title('Final budget')

# Remove spines on top and right sides
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
state, info = env.reset()
for _ in range(30):
    options = env.unwrapped.actions_available
    action = REINFORCE_agent.policy(state, greedy=True)
    
    s_prime, reward, terminated, truncated, info = env.step(action)
    print(f'Action: {action}', f'State: {state}', f'Reward: {round(reward,3)}', f'Terminated: {terminated}', truncated, info)

    if terminated or truncated:
        print(f'Final state: {s_prime}')
        print("============End of episode============")
        state, info = env.reset()
        break
    else:
        state = s_prime
env.close()

Save model

In [ ]:
experiment_folder = 's6-penalty2.5-edecay1-g0.999-storm0.6'
os.makedirs(f'agent_models/REINFORCENeuralAdvantage/{experiment_folder}', exist_ok=True)
torch.save(REINFORCE_agent.policy_net.state_dict(), f'agent_models/REINFORCENeuralAdvantage/{experiment_folder}/policy_net_weights-{int(n_episodes/1000)}k.pth')
torch.save(REINFORCE_agent.value_net.state_dict(), f'agent_models/REINFORCENeuralAdvantage/{experiment_folder}/value_net_weights-{int(n_episodes/1000)}k.pth')

Save data

In [ ]:
os.makedirs(f'data/REINFORCENeuralAdvantage/{experiment_folder}', exist_ok=True)
np.save(f'data/REINFORCENeuralAdvantage/{experiment_folder}/final_budget_queue-{int(n_episodes/1000)}k.npy', np.array(final_budget_queue))

# Montecarlo VFA

In [ ]:
nenv = gym.wrappers.RecordEpisodeStatistics(env, deque_size=n_episodes)
final_budget_queue = []
for episode in tqdm(range(n_episodes)):
    MC_vfa_agent.update(episode_number=episode)
    MC_vfa_agent.decay_epsilon()
    final_budget_queue.append(nenv.unwrapped.budget)

Statistics evaluation

In [ ]:
w = 1000
# rewards = conv(np.array(nenv.return_queue).flatten(), win=w)
# lengths = conv(np.array(nenv.length_queue).flatten(), win=w)
error = conv(np.array(MC_vfa_agent.training_error).flatten(), win=w)
final_budgets = conv(np.array(final_budget_queue), win=w)

In [ ]:
len(error), len(final_budgets)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4), ncols=2)
sns.lineplot(x=range(len(np.array(MC_vfa_agent.training_error).flatten())), y=np.array(MC_vfa_agent.training_error).flatten(), ax=ax[0], c='#acc700', linewidth=.2)
ax[0].set_title('Error')
sns.lineplot(x=range(len(final_budget_queue)), y=final_budget_queue, ax=ax[1], c='#acc700', linewidth=.2)
ax[1].set_title('Final budget')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4), ncols=2)
sns.lineplot(x=range(len(error)), y=error, ax=ax[0], c='#acc700', linewidth=.2)
ax[0].set_title('Error')
sns.lineplot(x=range(len(final_budgets)), y=final_budgets, ax=ax[1], c='#acc700', linewidth=.2)
ax[1].set_title('Final budget')
plt.tight_layout()
plt.show()

In [ ]:
MC_vfa_agent.w

In [ ]:
state, info = env.reset()
for _ in range(30):
    options = env.unwrapped.actions_available
    action = MC_vfa_agent.greedy_policy(state)
    
    s_prime, reward, terminated, truncated, info = env.step(action)
    print(f'Action: {action}', f'State: {state}', f'Reward: {round(reward,3)}', f'Terminated: {terminated}', truncated, info)

    if terminated or truncated:
        print(f'Final state: {s_prime}')
        print("============End of episode============")
        state, info = env.reset()
        break
    else:
        state = s_prime
env.close()

In [ ]:
experiment_folder = 'gaussian_delta-nowoolcost-15woolprice-200k'
os.makedirs(f'data/MCVFA/{experiment_folder}', exist_ok=True)

np.save(f'data/MCVFA/{experiment_folder}/weights.npy', MC_vfa_agent.w.detach().numpy())
np.save(f'data/MCVFA/{experiment_folder}/final_budget_queue.npy', np.array(final_budget_queue))
np.save(f'data/MCVFA/{experiment_folder}/training_error.npy', np.array(MC_vfa_agent.training_error).flatten())
# Split the 15M training error records in two chunks not to exceed GitHub limit of 100MB per file
# np.save('data/MCVFA/training_error_0k-750k.npy', MC_vfa_agent.training_error[: len(MC_vfa_agent.training_error) // 2])
# np.save('data/MCVFA/training_error_750k-1.5M.npy', MC_vfa_agent.training_error[len(MC_vfa_agent.training_error) // 2 : ])

# SARSA VFA

In [ ]:
nenv = gym.wrappers.RecordEpisodeStatistics(env, deque_size=n_episodes)
final_budget_queue = []
for episode in tqdm(range(n_episodes)):
    state, info = nenv.reset()
    done = False
    while not done:
        action = SARSA_vfa_agent.policy(state)
        # print(action)
        s_prime, reward, terminated, truncated, info = nenv.step(action=action)
        # update
        SARSA_vfa_agent.update(state, action, reward, s_prime)
        done = terminated or truncated
        state = s_prime
    SARSA_vfa_agent.decay_epsilon()
    final_budget_queue.append(nenv.unwrapped.budget)

Statistics evaluation

In [ ]:
w = 100
rewards = conv(np.array(nenv.return_queue).flatten(), win=w)
lengths = conv(np.array(nenv.length_queue).flatten(), win=w)
error = conv(np.array(SARSA_vfa_agent.training_error).flatten(), win=w)
final_budgets = conv(np.array(final_budget_queue), win=w)

In [ ]:
len(rewards),len(lengths),len(SARSA_vfa_agent.training_error)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 4), ncols=4)
ax[0].plot(range(len(nenv.return_queue)), nenv.return_queue, c='#acc700', linewidth=.4)
ax[0].set_title('Reward (Average)')
ax[1].plot(range(len(nenv.length_queue)), nenv.length_queue, c='#acc700', linewidth=.8)
ax[1].set_title('Episode Length')
ax[2].plot(range(len(SARSA_vfa_agent.training_error)), SARSA_vfa_agent.training_error, c='#acc700', linewidth=.2)
ax[2].set_title('Error')
ax[3].plot(range(len(final_budget_queue)), final_budget_queue, c='#acc700', linewidth=.2)
ax[3].set_title('Final budget')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 4), ncols=4)
sns.lineplot(x=range(len(rewards)), y=rewards, ax=ax[0], c='#acc700', linewidth=.4)
ax[0].set_title('Reward (Average)')
sns.lineplot(x=range(len(lengths)), y=lengths, ax=ax[1], c='#acc700', linewidth=.4)
ax[1].set_title('Episode Length')
sns.lineplot(x=range(len(error)), y=error, ax=ax[2], c='#acc700', linewidth=.2)
ax[2].set_title('Error')
sns.lineplot(x=range(len(final_budgets)), y=final_budgets, ax=ax[3], c='#acc700', linewidth=.2)
ax[3].set_title('Final budget')
plt.tight_layout()
plt.show()

In [ ]:
SARSA_vfa_agent.w